## IMPORTANTO BIBLIOTECAS

In [1]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn import preprocessing

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.efficientnet import EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

import os

## PARÂMETROS

In [2]:
percentege_test_size = 0.2
undersampling_size = 1300 # -1 para não fazer undersampling

# diretorio = '../dados classificacao imagens/animais-kaggle/'
# diretorio = '../dados classificacao imagens/dataset-documentos-full/'
diretorio = '../dados classificacao imagens/pneumonia/'
# diretorio = '../dados classificacao imagens/star-wars/'
# diretorio = '/home/carlos/Downloads/dados-docs/'
# diretorio = '../dados classificacao imagens/gatos-patos-caes-rapozas-coelhos/treino/'

### CARREGANDO DATASET

In [3]:
targets = [x[0].replace(diretorio,'') for x in os.walk(diretorio)]

targets = targets[1:len(targets)] # exclui o primeiro da lista pq é o próprio diretório

dados = []

for classe in targets:
    imagens = os.listdir(diretorio+classe)
    i = 0
    for img in imagens:
        i+=1
        
        if (not '.txt' in img) and (not 'segmentation' in img) and (not 'desktop.ini' in img):
            dados.append([diretorio+classe+'/'+img,classe])
        
        if(i == undersampling_size and undersampling_size != -1):
            break

dados = np.array(dados)

np.random.shuffle(dados)
            
df_dados = pd.DataFrame(dados,columns=['image','target'])

df_dados

,image,target
0,../dados classificacao imagens/pneumonia/NORMA...,NORMAL
1,../dados classificacao imagens/pneumonia/PNEUM...,PNEUMONIA
2,../dados classificacao imagens/pneumonia/PNEUM...,PNEUMONIA
3,../dados classificacao imagens/pneumonia/NORMA...,NORMAL
4,../dados classificacao imagens/pneumonia/NORMA...,NORMAL
...,...,...
2595,../dados classificacao imagens/pneumonia/NORMA...,NORMAL
2596,../dados classificacao imagens/pneumonia/PNEUM...,PNEUMONIA
2597,../dados classificacao imagens/pneumonia/NORMA...,NORMAL
2598,../dados classificacao imagens/pneumonia/NORMA...,NORMAL


## PREPARANDO DADOS

In [4]:
def getEmbedder():
    # return EfficientNetB0(weights='imagenet', include_top=False)
    # return EfficientNetB2(weights='imagenet', include_top=False)
    return EfficientNetB3(weights='imagenet', include_top=False)
    # return EfficientNetB4(weights='imagenet', include_top=False)
    # return EfficientNetB7(weights='imagenet', include_top=False)
    # return InceptionV3(weights='imagenet', include_top=False)

def embedding(embedder,img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    embed = image.img_to_array(img)
    embed = np.expand_dims(embed, axis=0)
    # embed = preprocess_input(embed)

    features = embedder.predict(embed)

    return features[0][0][0]

embedder = getEmbedder()

2022-09-27 20:27:36.458115: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-27 20:27:36.482675: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-27 20:27:36.482825: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-27 20:27:36.483428: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
# embedding(embedder,dados[0][0])

In [ ]:
target = df_dados['target'].array

array_features = []
for x in dados:
    array_features.append(embedding(embedder,x[0]))

scaler = preprocessing.StandardScaler()

array_features = scaler.fit_transform(array_features)

df_features = pd.DataFrame(array_features)

df_features

2022-09-27 20:27:38.455131: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-09-27 20:27:40.136806: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201


## REDUZINDO A DIMENSIONALIDADE

In [ ]:
model = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=15)
array_red = model.fit_transform(df_features) 

df_tsne = pd.DataFrame(array_red)

df_tsne['target'] = target

plt.rcParams['figure.figsize'] = [15, 10]

sns.scatterplot(data=df_tsne, x=df_tsne[0], y=df_tsne[1], hue=df_tsne['target'], palette="colorblind")

plt.show()

## TREINANDO E VALIDANDO O MODELO

In [ ]:
def getModel():
    return RandomForestClassifier(n_jobs=20)
    # return LogisticRegression()#n_jobs=20, solver='sag', multi_class='ovr')
    # return KNeighborsClassifier(n_neighbors=3,n_jobs=20)
    # return XGBClassifier()
    # return SGDClassifier(n_jobs=20,early_stopping=True,validation_fraction=0.05)

In [ ]:
X_treino, X_teste, y_treino, y_teste = train_test_split(df_features.values,target,test_size=percentege_test_size)

model = getModel().fit(X_treino, y_treino)

score = model.score(X_teste, y_teste)

score

## VALIDANDO DE FORMA MAIS ELABORADA

In [ ]:
scores = cross_val_score(getModel(),df_features.values,target,cv=10)

np.mean(scores)

## VISUALIZANDO A MATRIZ DE CONFUSÃO

In [ ]:
predicoes = cross_val_predict(getModel(),df_features.values,target,cv=10)

cm = confusion_matrix(target, predicoes)

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=model.classes_)
disp.plot()

plt.show()